In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pycountry

In [2]:
url = "https://www.cia.gov/the-world-factbook/field/suffrage/"
req = requests.get(url)

In [3]:
soup = BeautifulSoup(req.text, "html.parser")
soup

<!DOCTYPE html>
<html><head><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><style id="typography.js">html{font-family:sans-serif;-ms-text-size-adjust:100%;-webkit-text-size-adjust:100%}body{margin:0}article,aside,details,figcaption,figure,footer,header,main,menu,nav,section,summary{display:block}audio,canvas,progress,video{display:inline-block}audio:not([controls]){display:none;height:0}progress{vertical-align:baseline}[hidden],template{display:none}a{background-color:transparent;}a:active,a:hover{outline-width:0}abbr[title]{border-bottom:none;text-decoration:underline;text-decoration:underline dotted}b,strong{font-weight:inherit;font-weight:bolder}dfn{font-style:italic}h1{font-size:2em;margin:.67em 0}mark{background-color:#ff0;color:#000}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}img{

In [24]:
pb30_divs = soup.find_all("div", class_="pb30") # finding all the divs which contain the voting data

rows = []
for div in pb30_divs:
    try:
        country_h3 = div.find("h3")
        country_name = country_h3.get_text()
        full_text = div.get_text().replace(country_name, "").strip()

        # splitting the text into usable data
        suffrage_data = full_text.split(";")
        age = suffrage_data[0].replace(" years of age", "").strip()
        age = pd.to_numeric(age, errors="coerce") # errors="coerce" will turn any non-numeric values into NaN

        rows.append({
            "country": country_name,
            "age": age
        })
    except:
        pass

df = pd.DataFrame(rows)

manual_fixes = [
    {'country': 'Brazil', 'age': 16},
]

df = pd.concat([df, pd.DataFrame(manual_fixes)])

df = df.drop_duplicates(subset="country", keep="last")

df[df.age.isna()]

,country,age
8,Argentina,NaN
25,Bosnia and Herzegovina,NaN
29,Brunei,NaN
61,Ecuador,NaN
69,European Union,NaN
92,Holy See (Vatican City),NaN
94,Hong Kong,NaN
95,Hungary,NaN
115,Kuwait,NaN
122,Libya,NaN


In [35]:
def get_country(name):
    try:
        # first try a direct match
        country = pycountry.countries.get(name=name)
        if country:
            return country.alpha_3
        # if that failes, try a fuzzy match
        country = pycountry.countries.search_fuzzy(name)
        if country:
            return country.alpha_3
        return None
    except:
        return None

df['iso3'] = df['country'].apply(get_country)
df[df.iso3.isna()]


,country,age,iso3
14,"Bahamas, The",18.0,None
24,Bolivia,18.0,None
28,British Virgin Islands,18.0,None
29,Brunei,NaN,None
32,Burma,18.0,None
47,"Congo, Democratic Republic of the",18.0,None
48,"Congo, Republic of the",18.0,None
51,Cote d'Ivoire,18.0,None
54,Curacao,18.0,None
69,European Union,NaN,None


In [38]:
pycountry.countries.get(name="Laos")

In [9]:
div

<div class="az-filter__header fs-small pb30"><div><span class="label">Filter</span><div class="az-filter_mobile_display bold">All</div></div><div class="az-filter__mobile"><button class="az-filter__mobile-button" tabindex="0"><svg aria-labelledby="point-down-icon" class="icon icon-red" role="img" viewbox="0 0 30 30"><polygon fill-rule="nonzero" points="23.354 12.075 15.499 19.929 7.646 12.075 8.353 11.368 15.498 18.514 22.646 11.367 23.354 12.075"></polygon></svg></button></div></div>